In [1]:
import pandas as pd
import numpy as np
from simpledbf import Dbf5

In [2]:
dbf = Dbf5('./block_groups_2019.dbf')
df1 = dbf.to_dataframe()

In [3]:
# database -- census data
print(df1.shape)
df1.head()

(220740, 8)


,GEOID,NAMELSAD,ALAND,AWATER,INTPTLAT,INTPTLON,Y_LAT,X_LONG
0,010010201001,Block Group 1,4259428.0,28435.0,+32.4658320,-86.4896607,32.465832,-86.489661
1,010010201002,Block Group 2,5558385.0,0.0,+32.4858670,-86.4896713,32.485867,-86.489671
2,010010202001,Block Group 1,2058380.0,0.0,+32.4800823,-86.4749744,32.480082,-86.474974
3,010010202002,Block Group 2,1267300.0,5669.0,+32.4644350,-86.4697659,32.464435,-86.469766
4,010010203001,Block Group 1,3866515.0,9054.0,+32.4801750,-86.4607925,32.480175,-86.460792


In [4]:
# just the cols needed, padded
df = df1[['GEOID', 'Y_LAT', 'X_LONG']]
df.loc[:, 'GEOID'] = df.loc[:, 'GEOID'].apply(lambda x: str(x).zfill(12))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
# nhes ecpp data
ecpp1 = pd.read_csv('I:/NCES/NCES_Dev/NHES_BATTLE_MOVE/NHES 2019/1_NHES19 data processing/File Delivery/NCES_FINAL DELIVERY_012521/NHES19_RUF/NHES2019 ECPP data files – restricted-use/ecpp_ru_pert.csv', encoding='latin-1')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (25,54,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print(ecpp1.shape)
ecpp1.head()

(7092, 668)


,BASMID,RCVDATE,RCNOW,RCWEEK,RCTYPE,RCAGE,RCPLACE,RCTIME,RCDAYS,RCHRS,...,F_HVINTSPHO,F_HVINTCOM,F_CHLDNT,F_LRNCOMP,F_LRNTAB,F_LRNCELL,F_HHUNID,F_ZIPLOCL,F_RSTATE,F_ZCTA
0,20191000097,3192019,2,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1,20191000098,1212019,2,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,-1,-1,-1,0,0,0,0
2,20191000116,4152019,2,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,1,0,0,0,0
3,20191000180,5102019,1,1,1,73,2,2,1,3,...,0,0,0,0,0,0,0,0,0,0
4,20191000254,5282019,2,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,-1,-1,-1,0,0,0,0


In [7]:
# just the cols needed, padded
ecpp = ecpp1[['BASMID', 'CENBLGRP']]
ecpp.loc[:, 'CENBLGRP'] = ecpp.loc[:, 'CENBLGRP'].apply(lambda x: str(x).zfill(12))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
# merge census and ecpp together
d = pd.merge(ecpp, df, how='left', left_on='CENBLGRP', right_on='GEOID')
d

,BASMID,CENBLGRP,GEOID,Y_LAT,X_LONG
0,20191000097,270190907022,270190907022,44.838639,-93.528059
1,20191000098,420710129003,420710129003,40.080044,-76.078928
2,20191000116,245101205001,245101205001,39.309412,-76.616795
3,20191000180,160010019002,160010019002,43.593097,-116.269694
4,20191000254,250214401004,250214401004,42.004333,-71.362705
...,...,...,...,...,...
7087,20191225397,340057027003,340057027003,40.027869,-74.798999
7088,20191225427,121270909031,121270909031,28.906278,-81.312331
7089,20191225433,240054405002,240054405002,39.360803,-76.517601
7090,20191225461,480850305143,480850305143,33.153048,-96.672826


In [9]:
# check nulls
d['GEOID'].isnull().sum()

3

In [10]:
from geopy.distance import geodesic
from geopy.distance import distance

In [11]:
# library data
libs = pd.read_csv('./PLS_FY19_Outlet_pud19i.csv', encoding='latin-1')

In [12]:
d['FSCSKEY'] = [np.nan] * d.shape[0]
d['FSCS_SEQ'] = [np.nan] * d.shape[0]
d['DISTANCE'] = [np.nan] * d.shape[0]

In [13]:
# go through each row
mx = 50 #within 50 miles
for i, row in d.iterrows():
    if not np.isnan(row['Y_LAT']):
        coord1 = (row['Y_LAT'], row['X_LONG'])
        dst = distance(miles=mx*1.1)
        n = dst.destination(point=coord1, bearing=0)[0] #get latitude for north-bound
        e = dst.destination(point=coord1, bearing=90)[1] #get longitude for east-bound
        s = dst.destination(point=coord1, bearing=180)[0] #get latitude for south-bound
        w = dst.destination(point=coord1, bearing=-90)[1] #get latitude for west-bound
        # filter lat-lon to reduce compute time
        lib_filter = libs.loc[(libs['LATITUDE'] < n) & (libs['LATITUDE'] > s) & (libs['LONGITUD'] < e) & (libs['LONGITUD'] > w), :]
        
        near = []
        for j, wor in lib_filter.iterrows():
            coord2 = (wor['LATITUDE'], wor['LONGITUD'])
            dist = geodesic(coord1, coord2).miles
            if dist <= mx:
                near.append((dist, wor['FSCSKEY'], wor['FSCS_SEQ']))

        if len(near) > 0:
            min_tuple = min(near, key=lambda x:x[0])
            print(i, min_tuple)
            d.loc[i, 'DISTANCE'] = min_tuple[0]
            d.loc[i, 'FSCSKEY'] = min_tuple[1]
            d.loc[i, 'FSCS_SEQ'] = min_tuple[2]


0 (1.7622769871049664, 'MN0038', 2)
1 (1.6448926690363725, 'PA0195', 2)
2 (1.0521309259944098, 'MD0003', 2)
3 (1.2106768445666802, 'ID0005', 6)
4 (1.2801297248513386, 'MA0235', 2)
5 (1.7889178002774657, 'CA0265', 1)
6 (0.8155497211495587, 'WA0064', 3)
7 (1.2503674052907139, 'VA0047', 2)
8 (0.7705858553800883, 'OH0215', 4)
9 (1.7295472471291033, 'UT0019', 8)
10 (1.8007189769333922, 'NY0513', 2)
11 (2.0129044655930977, 'MT0051', 6)
12 (7.477925080198189, 'NC0048', 4)
13 (1.5032110069563012, 'OR0019', 6)
14 (2.0128648306223336, 'FL0092', 8)
15 (5.125901398317475, 'GA0006', 11)
16 (1.978228650796925, 'CA0104', 3)
17 (3.7215599668887345, 'OH0102', 2)
18 (0.1865138627169991, 'NC0026', 7)
19 (1.1290146621163446, 'TX0050', 16)
20 (0.5788543103184439, 'PA0385', 51)
21 (0.3593563598717898, 'CA0063', 39)
22 (1.4750507426400008, 'VA0026', 8)
23 (3.6777586738019337, 'OH0181', 2)
24 (0.4449984895873917, 'FL0103', 2)
25 (5.341683915863604, 'VA0086', 5)
26 (1.723523945613199, 'NC0061', 5)
27 (3.321907

222 (3.0546178631568175, 'PA9040', 1)
223 (0.5080059494540644, 'NY0004', 15)
224 (1.0980637502170212, 'CA0023', 7)
225 (0.7298501911251203, 'WA0064', 6)
226 (0.762944454647719, 'PA0459', 2)
227 (1.0459361583568731, 'WA0059', 12)
228 (2.7288319541439545, 'CA0099', 2)
229 (5.886221096530864, 'WI0121', 13)
230 (3.7013106711021493, 'TX0101', 4)
231 (1.509507980288362, 'VA0026', 25)
232 (0.5524291804148606, 'NY0327', 7)
233 (1.8559016505171413, 'AR0009', 9)
234 (0.6243881741345345, 'NY0562', 6)
235 (3.5962017587576613, 'AL0077', 2)
236 (2.2036123129988643, 'CO0001', 12)
237 (0.9604148504224006, 'AZ0088', 2)
238 (2.3365859713138803, 'DE0017', 2)
239 (2.3153212716082217, 'AZ0035', 10)
240 (2.6021760603535684, 'TN0072', 2)
241 (3.706794966116915, 'LA0037', 4)
242 (0.6227303367773162, 'MO0020', 6)
243 (2.6341787905916196, 'TX0217', 3)
244 (1.6598785904361344, 'MD0004', 23)
245 (0.607064602148041, 'OK0093', 13)
246 (2.911405382827347, 'AZ0034', 6)
247 (0.6370903463109748, 'MO0069', 2)
248 (0.517

438 (0.28375855714304127, 'CA0119', 2)
439 (1.5623004106557432, 'WA0063', 19)
440 (0.8439031633261213, 'OH0024', 2)
441 (1.8218898737207319, 'GA0054', 6)
442 (0.6441349295159172, 'VA0069', 6)
443 (5.5907676865217, 'AR0015', 4)
444 (1.0374528099249454, 'NC0021', 2)
445 (0.7845613870692751, 'OR0063', 13)
446 (2.295729775401317, 'TX0028', 2)
447 (1.2575614415172256, 'CA0062', 20)
448 (0.727603002299134, 'CA0062', 39)
449 (0.38647235195963675, 'MI0233', 21)
450 (1.853295062422824, 'NY0795', 1)
451 (6.196273723543492, 'WA0063', 19)
452 (1.291911351855424, 'MN0041', 41)
453 (0.13522021696177297, 'NJ0195', 10)
454 (2.789963687310038, 'MS0013', 7)
455 (0.4021876099976314, 'CA0063', 42)
456 (1.7568254488226571, 'ME0119', 2)
457 (1.0441306426773294, 'NY0720', 3)
458 (1.9781786659318275, 'CT0081', 2)
459 (0.870537782324487, 'TN0135', 12)
460 (1.3395628462892886, 'CO0034', 30)
461 (0.6705557018560587, 'TX0099', 12)
462 (1.3872275114740444, 'TX0041', 2)
463 (2.02433295352749, 'GA0013', 15)
464 (5.7

654 (0.9801416193553169, 'GA0022', 40)
655 (0.8742735116481263, 'TN0134', 35)
656 (2.937668827685229, 'NV0025', 6)
657 (0.8991148041858538, 'MN0106', 7)
658 (6.974970960895277, 'KY0110', 2)
659 (0.44552422005471903, 'PA0385', 4)
660 (1.7455450930832734, 'TN0062', 2)
661 (2.1752026819170722, 'IN0050', 6)
662 (0.38557881040895153, 'VA0005', 3)
663 (0.7861985067314826, 'MI0406', 2)
664 (9.174433959092774, 'PA0256', 2)
665 (1.1709272344347574, 'MO0014', 7)
666 (3.7570288256600395, 'NC0023', 4)
667 (1.2268746217805142, 'MD0002', 16)
668 (0.7733136267514436, 'IL0255', 2)
669 (0.2764301295844889, 'WI0049', 2)
670 (1.4851766881873028, 'NV0008', 21)
671 (6.415880570048578, 'WI0321', 2)
672 (4.592751488843386, 'WA0062', 31)
673 (1.362134173341274, 'OR0027', 19)
674 (5.542762980039573, 'SC0021', 7)
675 (1.2738581099877186, 'WA0067', 3)
676 (0.36738767059683075, 'CA0002', 4)
677 (1.0488128995974961, 'GA0023', 3)
678 (1.3830609791837831, 'OR0128', 2)
679 (0.9908076548251457, 'IL0049', 2)
680 (1.894

870 (0.9475807339375256, 'CA0102', 6)
871 (0.36360459595418093, 'PA0385', 396)
872 (0.45401846872017887, 'IN0091', 2)
873 (4.147853847101925, 'WV0007', 3)
874 (3.172767926309141, 'NY0020', 2)
875 (0.514272681124933, 'RI0010', 5)
876 (1.533612919995264, 'TX0228', 2)
877 (0.3247399195992544, 'CA0114', 16)
878 (0.3957036948028425, 'MN9030', 3)
879 (0.40753135227245735, 'CA0028', 11)
880 (0.9342854026373071, 'PA0385', 29)
881 (0.7404318820734375, 'IN0068', 11)
882 (4.431487005430205, 'VA0008', 6)
883 (0.5202417414179065, 'TX0101', 20)
884 (7.7961984509554965, 'VA0004', 9)
885 (7.454485638631527, 'MT0056', 2)
886 (0.13880151984167932, 'VA0005', 3)
887 (1.058033186524022, 'GA0017', 32)
888 (1.9945444794358884, 'MI0124', 2)
889 (4.202090836015806, 'GA0042', 2)
890 (1.355016477062959, 'TX0514', 2)
891 (6.155610813887002, 'NY0095', 2)
892 (1.6836353865309848, 'VA0026', 10)
893 (4.788338278392914, 'TX0703', 1)
894 (0.8910008653638829, 'FL0012', 37)
895 (2.356892903545344, 'FL0044', 10)
896 (3.40

...

6169 (1.1455598379029654, 'NJ0183', 2)
6170 (2.198563030231874, 'GA0007', 4)
6171 (0.4992479735502402, 'MA0035', 2)
6172 (0.5232975440525094, 'NY0592', 2)
6173 (1.0158453069085858, 'CA0007', 5)
6174 (1.2829017820181274, 'CA0113', 17)
6175 (1.8953622458907633, 'MN0044', 9)
6176 (1.85648094207688, 'CA0149', 2)
6177 (1.2716526790958425, 'MI0101', 2)
6178 (1.5670105812395752, 'UT0069', 4)
6179 (0.38419699152482784, 'FL0025', 5)
6180 (1.9623067886359196, 'WA0061', 6)
6181 (3.0406026558612913, 'TX0120', 2)
6182 (0.4067732866960083, 'OR9008', 1)
6183 (0.2961212516435759, 'OK0074', 5)
6184 (5.076627636637081, 'LA0005', 3)
6185 (6.362046889048162, 'TX0111', 22)
6186 (4.942956366458986, 'TX0394', 2)
6187 (1.1268206602391324, 'FL0044', 3)
6188 (2.0502149003663463, 'HI0001', 25)
6189 (0.4580711535280649, 'IN0030', 13)
6190 (0.7688665293188829, 'MN0155', 1)
6191 (1.6478601623196318, 'IA0510', 2)
6192 (0.7988710548562987, 'OH0009', 2)
6193 (2.987997385488973, 'MN0032', 24)
6194 (1.650886881392764, '

6379 (3.4987022331509436, 'MA0095', 2)
6380 (1.7620653205982477, 'IN0043', 8)
6381 (1.1777114039963217, 'PA0448', 4)
6382 (2.4520822977768617, 'FL0005', 3)
6383 (5.137711723260078, 'TN8019', 1)
6384 (1.673626189747532, 'NJ0160', 2)
6385 (1.647008034271634, 'SC8004', 5)
6386 (1.0334375803160198, 'IL0437', 2)
6387 (0.6216482129388249, 'FL0127', 5)
6388 (6.222894813850503, 'KY0094', 2)
6389 (2.9354945246079454, 'VA0014', 9)
6390 (1.2089351324657227, 'CA0063', 60)
6391 (2.3927127017090446, 'IL0467', 2)
6392 (1.5972430376316802, 'UT0061', 2)
6393 (0.39233377742657516, 'GA0025', 19)
6394 (1.9630621533875132, 'CA0126', 9)
6395 (3.552308327538161, 'GA0010', 3)
6396 (8.72108296621684, 'FL0019', 2)
6397 (2.4408926622172302, 'OH0130', 3)
6398 (0.4250333981202203, 'MI0035', 2)
6399 (0.4321061995106416, 'CA0133', 2)
6400 (0.516280994614752, 'CA0112', 26)
6401 (6.039955302250191, 'LA0055', 9)
6402 (1.7381025133011452, 'CA0040', 44)
6403 (2.136700881660604, 'TX0085', 2)
6404 (0.7735944546961703, 'OH0

6590 (2.3714667825930973, 'PA0353', 2)
6591 (0.865136204318764, 'KS0226', 2)
6592 (2.5372361513387416, 'WA0059', 12)
6593 (2.9314079085043656, 'IL0556', 2)
6594 (3.7801633120865463, 'FL0035', 44)
6595 (5.075777346360083, 'KY0011', 2)
6596 (0.7488676030915473, 'IL0098', 8)
6597 (5.07430014595267, 'VA0036', 8)
6598 (2.7917712737428246, 'VA0014', 3)
6599 (1.2620738583597477, 'IA0085', 2)
6600 (0.46213261222833335, 'CO0130', 6)
6601 (2.0841907040791803, 'SC0021', 11)
6602 (1.2812489413467332, 'IL0638', 2)
6603 (2.3068787747619997, 'VA0028', 4)
6604 (2.3849643080792986, 'NE0256', 2)
6605 (1.529887183285246, 'FL0012', 41)
6606 (0.8361109803456909, 'CA0016', 2)
6607 (0.6682747889009278, 'MA0116', 2)
6608 (0.9211802032798364, 'WI0103', 3)
6609 (5.146728954084388, 'TX0315', 2)
6610 (1.5289736410990744, 'NJ0075', 2)
6611 (0.5298263930538327, 'PA0447', 9)
6612 (1.1780941801942648, 'CO0085', 2)
6613 (0.17809022182790238, 'TX0095', 2)
6614 (2.073826021390015, 'CA0038', 7)
6615 (0.6808704293804736, 

6803 (0.5976109595092681, 'FL0039', 22)
6804 (1.5172975957470236, 'TX0221', 3)
6805 (0.3944182993587774, 'NY0343', 2)
6806 (6.658153212478343, 'NM0125', 1)
6807 (4.266042020910221, 'NY0626', 2)
6808 (2.6270352867433395, 'NC0063', 29)
6809 (0.4866714303578282, 'FL0025', 6)
6810 (1.042618558810241, 'CA0110', 3)
6811 (0.3177393425209165, 'WA0066', 3)
6812 (2.819959006673767, 'GA0025', 14)
6813 (0.7979720689578463, 'PA0302', 2)
6814 (1.5670378102325837, 'IN0210', 42)
6815 (1.084226757498642, 'GA0049', 5)
6816 (2.169191450262633, 'FL0023', 8)
6817 (1.1686353373221439, 'NY0669', 3)
6818 (1.2994817028543804, 'PA0448', 7)
6819 (4.187257401649472, 'GA0053', 2)
6820 (2.2991831572392836, 'MI0106', 2)
6821 (3.289535318020678, 'TX0443', 8)
6822 (3.0997897597043877, 'CA0063', 46)
6823 (1.5093518287943, 'CA0137', 20)
6824 (1.9522878232981093, 'IA0147', 2)
6825 (1.2190528994412912, 'NJ0168', 2)
6826 (1.355821489000299, 'OH0057', 7)
6827 (12.939004701157362, 'OK0001', 2)
6828 (1.4305746801708117, 'MI02

7014 (1.400864585598795, 'CA0140', 10)
7015 (4.040203834883188, 'NC0023', 4)
7016 (4.6867777326406115, 'TX0485', 3)
7017 (10.618384127080963, 'KS0131', 2)
7018 (1.645552137732992, 'TX0263', 19)
7019 (3.4799699598591003, 'MI0028', 2)
7020 (2.1595238586315406, 'NM0002', 22)
7021 (2.3797745506241204, 'GA0042', 13)
7022 (2.7077525763702814, 'WA0069', 18)
7023 (0.14735428569751716, 'NY0778', 24)
7024 (5.472202521404979, 'AZ0064', 22)
7025 (2.765758078620976, 'GA0025', 13)
7026 (0.6928781447748039, 'NY0562', 44)
7027 (5.5447301730741145, 'PA0101', 4)
7028 (0.6579804805755981, 'NY0360', 2)
7029 (1.1721022500567277, 'IN0157', 2)
7030 (1.509743789851035, 'CO0046', 2)
7031 (0.44230497233537075, 'NY0562', 48)
7032 (1.3923878974768307, 'CT0088', 2)
7033 (1.8990542570930313, 'CA0040', 11)
7034 (1.23279174627894, 'CA0084', 13)
7035 (1.3599690267746531, 'NY0104', 2)
7036 (4.4938345314342, 'ND0091', 2)
7037 (2.9723369672387348, 'PA0013', 2)
7038 (0.952690335187848, 'IL0018', 2)
7039 (3.863270137978807

In [14]:
d

,BASMID,CENBLGRP,GEOID,Y_LAT,X_LONG,FSCSKEY,FSCS_SEQ,DISTANCE
0,20191000097,270190907022,270190907022,44.838639,-93.528059,MN0038,2.0,1.762277
1,20191000098,420710129003,420710129003,40.080044,-76.078928,PA0195,2.0,1.644893
2,20191000116,245101205001,245101205001,39.309412,-76.616795,MD0003,2.0,1.052131
3,20191000180,160010019002,160010019002,43.593097,-116.269694,ID0005,6.0,1.210677
4,20191000254,250214401004,250214401004,42.004333,-71.362705,MA0235,2.0,1.280130
...,...,...,...,...,...,...,...,...
7087,20191225397,340057027003,340057027003,40.027869,-74.798999,NJ0070,2.0,1.174248
7088,20191225427,121270909031,121270909031,28.906278,-81.312331,FL0099,3.0,0.661035
7089,20191225433,240054405002,240054405002,39.360803,-76.517601,MD0004,18.0,1.434834
7090,20191225461,480850305143,480850305143,33.153048,-96.672826,TX0171,3.0,2.394086


In [15]:
d['DISTANCE'].isnull().sum()

3

In [16]:
# save
d.to_csv('I:/NCES/NCES_Dev/NHES_BATTLE_MOVE/NHES PLS analysis/ecpp_ru_pert_census19_libs50.csv', index=False)

In [17]:
d['DISTANCE'].max()

36.12513509001617